In [1]:
import torch_geometric as tg
import torch 
from utils.data import build_data, get_target, get_neighbors
from utils.training import get_model, train

import pandas as pd
import numpy as np

In [2]:
data_path = 'database.pkl'
df = pd.read_pickle(data_path)
df.dropna(inplace=True)

In [3]:
df.head()

,comp,structure,Freq_meV,a2F,PhFreq_meV,Tot_PhDOS,Site_Proj_PhDOS,lambda,w_log,w_sq,...,PhFreq_meV_dense,Tot_PhDOS_dense,Site_Proj_PhDOS_dense,Tot_eDOS_eng_meV,Tot_eDOS,Site_proj_eDOS_eng_meV,Site_proj_eDOS,Ph_2x2x2_interpolated_Freq_meV,Ph_2x2x2_interpolated_Tot_DOS,Ph_2x2x2_interpolated_Site_Proj_DOS
0,Cs,"(Atom('Cs', [0.0, 0.0, 0.0], index=0))","[0.2500020894237172, 0.3500026530791246, 0.450...","[0.000393028, 0.00107847, 0.00229214, 0.004184...","[-6.085040398332379e-08, 0.10000080571388359, ...","[0.0, 0.0020322211624833066, 0.008128885254687...","[[0.0, 0.0020321934853404634, 0.00812925787376...",0.200493,2.004419,6.257714,...,"[-6.085040398332379e-08, 0.10000080571388359, ...","[0.0, 0.0012481590493140242, 0.004758605329721...","[[0.0, 0.0012481423315464227, 0.00475858800768...","[-59.90000000000029, -58.89999999999995, -57.9...","[1.2210457884862193e-05, 1.2210457884862193e-0...","[-59.90000000000029, -58.89999999999995, -57.9...","[[1.117835043145045e-05, 1.1270872400968928e-0...","[0.0, 0.10000086656411722, 0.20000173312823444...","[0.0, 0.002723499688694547, 0.0108940020350333...","[[0.0, 0.0027234909279048826, 0.01089412502242..."
1,AuInPdY,"(Atom('Y', [0.0, 0.0, 0.0], index=0), Atom('In...","[0.2500020894237172, 0.3500026530791246, 0.450...","[0.0030494, 0.00836756, 0.0177841, 0.03247, 0....","[-2.6987919082213373e-07, 0.10000059668510464,...","[0.0, 0.0008684808446008167, 0.003473923532455...","[[0.0, 0.0001485995109299599, 0.00057976715396...",2.387674,2.187430,24.628668,...,"[-2.6987919082213373e-07, 0.10000059668510464,...","[0.0, 3.643101440717545e-05, 0.000145724082212...","[[0.0, 6.383229751421453e-06, 2.55177557902848...","[-59.99999999999872, -59.00000000000105, -57.9...","[2.3458147467037518e-05, 2.3458147467037518e-0...","[-59.99999999999872, -59.00000000000105, -57.9...","[[9.043179593508906e-06, 9.04305210357702e-06,...","[-5.031994663340724e-07, 0.10000036336409018, ...","[0.0, 0.0001446230318617502, 0.000578493267341...","[[0.0, 2.5177389997772625e-05, 9.9996566245638..."
2,Pd2SnY,"(Atom('Y', [-3.344924900585068, 3.344924900585...","[0.2500020894237172, 0.3500026530791246, 0.450...","[0.000446282, 0.0012246, 0.00260272, 0.0047520...","[-1.870816176469976e-07, 0.10000067948303153, ...","[0.0, 0.00029649550326095366, 0.00118598279430...","[[0.0, 6.174332262652135e-05, 0.00024316796868...",2.142195,3.377658,33.592206,...,"[-1.870816176469976e-07, 0.10000067948303153, ...","[0.0, 4.442308908471899e-05, 0.000177692694081...","[[0.0, 9.386675576437653e-06, 3.75176663613656...","[-59.00000000000105, -57.99999999999983, -57.0...","[2.444904947840698e-05, 2.444904947840698e-05,...","[-59.00000000000105, -57.99999999999983, -57.0...","[[6.776663285157005e-06, 6.776396082430465e-06...","[-3.5685336069797726e-07, 0.10000050971014814,...","[0.0, 6.652902586054768e-05, 0.000266116923297...","[[0.0, 1.3975967897277166e-05, 5.5539309176275..."
3,Hg3Tl,"(Atom('Tl', [0.0, 0.0, 0.0], index=0), Atom('H...","[0.2500020894237172, 0.3500026530791246, 0.450...","[6.38383e-05, 0.000175172, 0.000372305, 0.0006...","[-1.1853092944257293e-07, 0.10000074803268756,...","[0.0, 0.0009253369606325421, 0.003701350212266...","[[0.0, 0.00023463462725051366, 0.0009387482130...",0.875469,3.744005,21.080808,...,"[-1.1853092944257293e-07, 0.10000074803268756,...","[0.0, 0.000958242776325254, 0.0038329728614110...","[[0.0, 0.00024280501937882723, 0.0009706877518...","[-59.499999999999886, -58.50000000000044, -57....","[1.0029044003269802e-05, 1.012834146864871e-05...","[-59.499999999999886, -58.50000000000044, -57....","[[5.226121921433394e-06, 5.236053653920593e-06...","[-6.10494434250373, -6.004943475939613, -5.904...","[0.0, 0.44470284544261407, 0.5154988295465505,...","[[0.0, 0.0018224894425604506, 0.00330017704554..."
4,AuLiSb,"(Atom('Li', [0.0, 0.0, 0.0], index=0), Atom('S...","[0.2500020894237172, 0.3500026530791246, 0.450...","[8.20164e-05, 0.000225053, 0.00047832, 0.00087...","[-2.591243543263685e-07, 0.10000060743949912, ...","[0

In [4]:
r_max = 4
df['target'] = df.apply(get_target,axis=1)
df['formula'] = df['structure'].map(lambda x: x.get_chemical_formula())
df['species'] = df['structure'].map(lambda x: list(set(x.get_chemical_symbols())))
species = sorted(list(set(df['species'].sum())))

df['data'] = df.apply(build_data,embed_ph_dos=True,embed_e_dos=False,fine=False,r_max = r_max,axis=1)


In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

out_dim = len(df.iloc[0]['target'])
in_dim = len(df.iloc[0].data.x[0])
em_dim = 64

In [6]:
init_dict_base = dict(in_dim=118+51,                           
    em_dim=em_dim,                        
    irreps_in=str(em_dim)+"x0e",          
    irreps_out=str(out_dim)+"x0e",        
    irreps_node_attr=str(em_dim)+"x0e",   
    layers=2,                             
    mul=32,                               
    lmax=1,                               
    max_radius=r_max,      
    num_neighbors=get_neighbors(df,df.index).mean(),      
    reduce_output=True,
    p=0.0
)

In [7]:
max_iter = 50

In [8]:
loss_fn = torch.nn.MSELoss()
loss_fn_mae = torch.nn.MSELoss()

In [9]:
folds = range(10)
for i in folds:
    idx_train = pd.Index(np.loadtxt(f'indices/idx_train_V2_{i}.txt'))
    idx_valid = pd.Index(np.loadtxt(f'indices/idx_valid_V2_{i}.txt'))
    
    dataloader_train = tg.loader.DataLoader(df.iloc[df.index.isin(idx_train)]['data'].values, batch_size=652)
    dataloader_valid = tg.loader.DataLoader(df.iloc[df.index.isin(idx_valid)]['data'].values, batch_size=164)
    name = f'models/CPD_test_{i}'
    
    run_name = f'{name}'
    print(name)

    model, opt, scheduler = get_model(init_dict_base,lr = 0.005,wd=0,device=device)

    train(model, opt, dataloader_train, dataloader_valid,
          loss_fn, loss_fn_mae, run_name,
          max_iter=max_iter, scheduler=scheduler, device=device)

test_0
Validation loss decreased (inf --> 0.010311).  Saving model ...
Validation loss decreased (0.010311 --> 0.010311).  Saving model ...
Validation loss decreased (0.010311 --> 0.010310).  Saving model ...
Validation loss decreased (0.010310 --> 0.010310).  Saving model ...
Validation loss decreased (0.010310 --> 0.010309).  Saving model ...
Validation loss decreased (0.010309 --> 0.010308).  Saving model ...
Validation loss decreased (0.010308 --> 0.010306).  Saving model ...
Validation loss decreased (0.010306 --> 0.010302).  Saving model ...
Validation loss decreased (0.010302 --> 0.010297).  Saving model ...
Validation loss decreased (0.010297 --> 0.010289).  Saving model ...
Validation loss decreased (0.010289 --> 0.010276).  Saving model ...
Validation loss decreased (0.010276 --> 0.010253).  Saving model ...
Validation loss decreased (0.010253 --> 0.010212).  Saving model ...
Validation loss decreased (0.010212 --> 0.010140).  Saving model ...
Validation loss decreased (0.010


KeyboardInterrupt

